In [1]:
import pandas as pd

text = pd.read_csv("민원데이터.csv", encoding="utf-8")

In [2]:
text.head()

,제목,질문내용,답변내용,등록일시,최상위기관명,분야코드,분야명,담당부서명
0,의성군 귀농귀촌 정책 문의,의성군 귀농귀촌 정책 문의,1. 우선 저희 의성군으로 귀농·귀촌하고자 하셔서 감사합니다.\n \n2. 귀농·귀...,2021-08-27 오후 3:31:44,경상북도 의성군,20010010,행정/자치/안전,농업기술센터
1,불법주정차 신고합니다,5대 불법주정차 과태료 부과 대상,1. 귀하의 가정에 평안함과 행복을 기원합니다. 귀하께서 국민신문고로 요청하신 사항...,2021-08-27 오후 1:49:33,경상북도 의성군,20010010,행정/자치/안전,일자리창출과
2,아동수당 지급 대상,아동수당 지급 대상이 궁금합니다.,1. 안녕하십니까? 귀하께서 문의하신 질의에 대한 답변을 다음과 같이 알려드립니다....,2021-08-27 오후 4:19:51,경상북도 의성군,20010010,행정/자치/안전,복지과
3,광견병 예방 접종 관련 문의,광견병 예방접종 질문사항.\n1. 의성군에서 광견병 접종 예방 계획이 있으신지요?...,안녕하십니까?\n1. 귀하께서 국민신문고를 통해 신청하신 민원에 대한 검토 결과를 ...,2021-08-27 오후 4:18:09,경상북도 의성군,20010010,행정/자치/안전,농축산과
4,아동수당 지급 금액과 지급 일자,아동수당 지급 금액과 지급 일자가 궁금합니다.,1. 안녕하십니까? 귀하께서 문의하신 질의에 대한 답변을 다음과 같이 알려드립니다....,2021-08-27 오후 4:24:24,경상북도 의성군,20010010,행정/자치/안전,복지과


In [3]:
text = text.dropna()

In [4]:
text.isna().any().any()

False

In [5]:
from konlpy.tag import Okt
tag = Okt()

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [6]:
def tokenizer(str):
    return [token for token in tag.nouns(str) if len(token) > 1]

In [ ]:
#count vector 새로 생성
vec = CountVectorizer(tokenizer=tokenizer, decode_error ='ignore', 
                      max_df=0.5, max_features = 1000) #너무 자주 나타나는 단어는 제외
text_CV = vec.fit_transform(text['질문내용'])

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d: " % topic_idx, end='')
        print(", ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        #print(", ".join([feature_names[i]+'('+str(topic[i])+')' for i in topic.argsort()[:-n_top_words - 1:-1]]))
        # 위 slicing에서 맨 뒤 -1은 역순을 의미, 역순으로 했을 때 처음부터 n_top_words까지
    print()

In [ ]:
#topic modeling 수행
lda = LatentDirichletAllocation(n_components = 50, max_iter=25, 
                                topic_word_prior= 0.1, doc_topic_prior=1.0,
                                learning_method='batch',
                                n_jobs= -1,
                                random_state=0)
news_topics = lda.fit_transform(text_CV)
print_top_words(lda,vec.get_feature_names(), 10)

#### 내용을 보니 불용어 처리가 필요해보임!!

In [ ]:
from konlpy.tag import Okt
tokenizer = Okt()

nouns_data = []
for i in text['질문내용'] :
    nouns = tokenizer.nouns(i)
    nouns_data.append(nouns)

In [ ]:
nouns_data[1]

In [ ]:
noun_to_sent = []
for i in nouns_data : 
    noun_to_sent.append([' '.join(i)])

In [ ]:
noun_to_sent[0]

In [ ]:
nouns_list = sum(nouns_data,[])

In [ ]:
from collections import Counter
count = Counter(nouns_list)
most_word = count.most_common(5000)

In [ ]:
word = []
count = []
for i in most_word:
    word.append(i[0])
    count.append(i[1])

In [ ]:
count_excel = pd.DataFrame()
count_excel["단어"] = word
count_excel["빈도"] = count
count_excel.head()

In [ ]:
count_excel.to_csv('word_count.csv')

In [ ]:
stop_data = pd.read_csv("word_count.csv")
stop_data[0:3]

In [ ]:
stop_words = list(stop_data["단어"][stop_data.빈도>2000])

In [ ]:
stop_words

In [ ]:
clean_data = [list(filter(lambda z: z not in stop_words, x)) for x in nouns_data]

In [ ]:
clean_data[0]

In [ ]:
noun_to_sent = []
for i in clean_data : 
    noun_to_sent.append(' '.join(i))

In [ ]:
noun_to_sent[0]

In [ ]:
clean_df = pd.DataFrame() 
clean_df["contents"]  = noun_to_sent

In [ ]:
clean_df["contents"][0] 

In [ ]:
clean_df

In [ ]:
from konlpy.tag import Okt
tag = Okt()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
vec = CountVectorizer(tokenizer=tokenizer, decode_error ='ignore', 
                      max_df=0.5, max_features = 1000) #너무 자주 나타나는 단어는 제외
text_stop_cv = vec.fit_transform(clean_df['contents'])

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d: " % topic_idx, end='')
        print(", ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        #print(", ".join([feature_names[i]+'('+str(topic[i])+')' for i in topic.argsort()[:-n_top_words - 1:-1]]))
        # 위 slicing에서 맨 뒤 -1은 역순을 의미, 역순으로 했을 때 처음부터 n_top_words까지
    print()

In [ ]:
#topic modeling 수행
lda = LatentDirichletAllocation(n_components = 50, max_iter=25, 
                                topic_word_prior= 0.1, doc_topic_prior=1.0,
                                learning_method='batch',
                                n_jobs= -1,
                                random_state=0)
news_topics = lda.fit_transform(text_stop_cv)
print_top_words(lda,vec.get_feature_names(), 10)